<a href="https://colab.research.google.com/github/xbadiam/Portfolio/blob/main/Forecasting/Bike_Sharing/GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Contexto

Los sistemas de bicicletas compartidas, también conocidos como sistemas de bicicletas públicas, facilitan la disponibilidad automática de bicicletas para que sean utilizadas temporalmente como medio de transporte. La mayoría de estos sistemas permiten recoger una bicicleta y devolverla en un punto diferente (estaciones o dockers), para que el usuario solo necesite tener la bicicleta en su posesión durante el desplazamiento. Uno de los principales retos en la gestión de estos sistemas es la necesidad de redistribuir las bicicletas para intentar que, en todas las estaciones, haya bicicletas disponibles a la vez que espacios libres para devoluciones.

A diferencia de otros servicios de transporte como el autobús o el metro, la duración del viaje y la posición de salida y llegada se registran explícitamente en estos sistemas. Esta característica convierte al sistema de bicicletas compartidas en una red virtual de sensores que puede utilizarse para medir la movilidad en la ciudad. Por lo tanto, se espera que la mayoría de los eventos importantes en la ciudad puedan detectarse mediante el monitoreo de estos datos.

Con el objetivo de mejorar la planificación y ejecución de la distribución de las bicicletas, se plantea crear un modelo capaz de predecir el número de usuarios para las siguientes 36 horas. De esta forma, a las 12h de cada día, la compañía encargada de gestionar las estaciones de alquiler podrá conocer la demanda prevista el resto del día (12 horas) y el siguiente día (24 horas).

### 1.2. Descripción de los datosEnendiendo los datos

* **instant**: Índice de registros
* **dteday**: Fecha
* **season**: Estación del año (1:springer, 2:summer, 3:fall, 4:winter)
* **yr**: Año (0: 2011, 1:2012)
* **mnth**: mes (1 a 12)
* **hr** houras del día (0 to 23)
* **holiday**: Indica si el día del tiempo es festivo o no.
* **weekday**: día de la semana
* **workingday**: Si el día no es fin de semana ni festivo es 1, en caso contrario es 0.
* **weathersit**:
  * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
  * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain +   Scattered clouds
  * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **atemp**: Temperatura normalizada en grados Celsius. Los valores se derivan de(t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
* **atemp**: Sensación temperatura normalizada en grados Celsius. Los valores se derivan de(t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
	* hum: Normalized humidity. The values are divided to 100 (max)
	* windspeed: Normalized wind speed. The values are divided to 67 (max)
	* casual: count of casual users
	* registered: count of registered users
	* cnt: count of total rental bikes including both casual and registered

In [2]:
! git clone https://github.com/xbadiam/Portfolio.git

Cloning into 'Portfolio'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 174 (delta 2), reused 0 (delta 0), pack-reused 162 (from 1)
Receiving objects: 100% (174/174), 14.95 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
import pandas as pd
import numpy as np

## 4. Lectura de los datos

In [4]:
hour_data = pd.read_csv('/content/Portfolio/Forecasting/Bike_Sharing/hour.csv')
hour_data.tail()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61
17378,17379,2012-12-31,1,1,12,23,0,1,1,1,0.26,0.2727,0.65,0.1343,12,37,49


## 5. Exploración y gestión de los datos

In [5]:
hour_data.shape

(17379, 17)

In [6]:
hour_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [7]:
# Verificamos si existen valores nulos
hour_data.isnull().sum()

,0
instant,0
dteday,0
season,0
yr,0
mnth,0
hr,0
holiday,0
weekday,0
workingday,0
weathersit,0


In [8]:
# Identificamos los duplicados
print(hour_data.duplicated().sum())

0


In [9]:
# Renombramos el nombre de las columnas
hour_data = hour_data.rename(columns={'dteday': 'date_time',
                                      'weathersit': 'weather',
                                      'yr': 'year',
                                      'mnth': 'month',
                                      'hr': 'hour',
                                      'hum': 'humidity',
                                      'cnt': 'users'})

In [10]:
# Se renombran los niveles de la variable meteorológica. Solo hay cuatro valores
# de heavy rain, así que se consideran como rain.
hour_data['weather'] = hour_data['weather'].replace({1: 'clear',
                                                     2: 'mist',
                                                     3: 'rain',
                                                     4: 'rain'})
